# Decepticons: Corrupted Transformers Breach Privacy in Federated Learning for Language Models

This notebook shows an example for the threat model and attack described in "Decepticons: Corrupted Transformers Breach Privacy in Federated Learning for Language Models
". This example deviates from the other "honest-but-curious" server models and investigates a malicious server that may send malicious server updates. The attack succeeds for a range of common transformer architectures and works merely by sending a single malicious query to the user model.

In this notebook, we attack the (small) GPT-2 model (120mil parameters) and show an example for custom text input (The wikipedia page on decepticons, up to the "the dark side of the moon".)



Paper URL: https://arxiv.org/abs/2201.12675

### Abstract:
A central tenet of Federated learning (FL), which trains models without centralizing user data, is privacy. However, previous work has shown that the gradient updates used in FL can leak user information. While the most industrial uses of FL are for text applications (e.g. keystroke prediction), nearly all attacks on FL privacy have focused on simple image classifiers. We propose a novel attack that reveals private user text by deploying malicious parameter vectors, and which succeeds even with mini-batches, multiple users, and long sequences. Unlike previous attacks on FL, the attack exploits characteristics of both the Transformer architecture and the token embedding, separately extracting tokens and positional embeddings to retrieve high-fidelity text. This work suggests that FL on text, which has historically been resistant to privacy attacks, is far more vulnerable than previously thought.

### Startup

In [ ]:
try:
    import breaching
except ModuleNotFoundError:
    # You only really need this safety net if you want to run these notebooks directly in the examples directory
    # Don't worry about this if you installed the package or moved the notebook to the main directory.
    import os; os.chdir("..")
    import breaching
    
import torch
%load_ext autoreload
%autoreload 2

# Redirects logs directly into the jupyter notebook
import logging, sys
logging.basicConfig(level=logging.INFO, handlers=[logging.StreamHandler(sys.stdout)], format='%(message)s')
logger = logging.getLogger()

### Initialize cfg object and system setup:

This will load the full configuration object. This includes the configuration for the use case and threat model as `cfg.case` and the hyperparameters and implementation of the attack as `cfg.attack`. All parameters can be modified below, or overriden with `overrides=` as if they were cmd-line arguments.

In [ ]:
cfg = breaching.get_config(overrides=["attack=decepticon", "case=10_causal_lang_training", 
                                     "case/server=malicious-transformer"])
          
device = torch.device('cpu')
torch.backends.cudnn.benchmark = cfg.case.impl.benchmark
setup = dict(device=device, dtype=getattr(torch, cfg.case.impl.dtype))
setup

### Modify config options here

You can use `.attribute` access to modify any of these configurations for the attack, or the case:

In [ ]:
cfg.case.user.num_data_points = 8 # How many sentences?
cfg.case.user.user_idx = 1 # From which user?
cfg.case.data.shape = [512] # This is the sequence length

cfg.case.server.provide_public_buffers = True # Send server signal to disable dropout
cfg.case.server.has_external_data = True  # Not strictly necessary, but could also use random text (see Appendix)
cfg.case.data.tokenizer = "gpt2"
cfg.case.model = "gpt2" # Could also choose "gpt2S" which contains ReLU activations
cfg.case.server.pretrained = False

## Attack hyperparameters:

# Server side:
cfg.case.server.param_modification.v_length = 32 # Length of the sentence component
cfg.case.server.param_modification.eps = 1e-16
cfg.case.server.param_modification.measurement_scale=1e12 # Circumvent GELU
cfg.case.server.param_modification.imprint_sentence_position = 0
cfg.case.server.param_modification.softmax_skew = 1e8
cfg.case.server.param_modification.sequence_token_weight = 1


# Attacker side:

# this option requires installation of `k-means-constrained` which can be tricky:
# If this doesn't work for you, falling back to "dynamic-threshold" is still a decent option.
cfg.attack.sentence_algorithm = "k-means" 
cfg.attack.token_strategy="embedding-norm" # no decoder bias in GPT
cfg.attack.embedding_token_weight=0.25 # This can improve performance slightly for long sequences

### Instantiate all parties

The following lines generate "server, "user" and "attacker" objects and print an overview of their configurations.

In [ ]:
user, server, model, loss_fn = breaching.cases.construct_case(cfg.case, setup)
attacker = breaching.attacks.prepare_attack(server.model, server.loss, cfg.attack, setup)
breaching.utils.overview(server, user, attacker)

### Simulate an attacked FL protocol

This exchange is a simulation of a single query in a federated learning protocol. The server sends out a `server_payload` and the user computes an update based on their private local data. This user update is `shared_data` and contains, for example, the parameter gradient of the model in the simplest case. `true_user_data` is also returned by `.compute_local_updates`, but of course not forwarded to the server or attacker and only used for (our) analysis.

In [ ]:
server_payload = server.distribute_payload()

# Text from tranformers wikipedia page

In [ ]:
text = """Transformers: Generation 1
The Decepticons are from a race called Transformers - robots that can change into vehicles, mechanical devices and even animal forms. They waged war for eons on their home planet of Cybertron against the heroic Autobots. Both factions needed supplies, and select numbers from each side intended to get them. The Autobots did not wish to fight the Decepticons, but they fought in space anyway. When the Decepticons board the Autobots' ship it takes a beating and crashes onto Primitive Earth and knocks everyone unconscious. Millions of years later, in 1984 a volcano eruption causes a probe to give them all new forms which come from vehicles in the surrounding area. After the featured film The Transformers: The Movie, Megatron was reformed as Galvatron, and Optimus Prime was replaced for a time by Rodimus Prime, only to return later on.

Members
Unlike the Autobots, whose leader is a Prime bearing a Matrix, the Decepticons are led by the most powerful of their ranks (usually Megatron). This tends to cause some conflict, given how generally every Decepticon thinks that they're the most powerful. Also, the Decepticons are not exactly the most compassionate beings in the universe, but not all fight for greed. More than a few have a sense of honor, while others believe that Cybertron would be better protected by aggressive expansion. Ultimately, the Decepticons desire to protect their homeworld, even if it is at the expense of others. Other leaders include Galvatron, Shockwave, Soundwave, Scorponok, Devil Z, Deathsaurous, Starscream, Thunderwing, Jiaxus, Ratbat, and Bludgeaon. Some continuities also has Liege Maximo and The Fallen as the ancestor of the Decepticon faction.

Hasbro toyline
Hasbro launched the Transformers toy line with eighteen different Autobot characters, all of whom transformed into automobiles, while the ten distinct Decepticons, (seven packages as three came two to a box/pack) were weapons, aircraft and communications equipment. Many of the Transformers were based on Takara designs. Optimus Prime was among the first Transformers released from Hasbro in 1984. The character listing/mini-poster that came inside Transformer packaging identified him as "Autobot Commander", as contrasted with Megatron's title of "Decepticon Leader".

Marvel Comics
Main article: The Transformers (Marvel Comics)
In the Marvel comic continuity, it was established that the Decepticons came in and out of the gladiator pits on Cybertron. Early on, the leadership of the Decepticons constantly changed due to their tendency to kill and replace their leaders. It would not be until a lowly Decepticon affiliated gladiator named Megatron assumed the throne that the Decepticons became a true threat to the rest of the population of Cybertron.

While Megatron was the primary leader, other Decepticons had prominent roles as leaders of the group. Shockwave, Scorponok, Bludgeon, and Ratbat all led the Decepticons, often overthrowing Megatron in the process or taking over the group during the period when Megatron was believed dead.

In the G2 Marvel series (which continues from the G1 Marvel series), it is stated that the Decepticon forces on Cybertron underwent a massive split in the wake of Megatron's disappearance when Megatron, Optimus Prime, and their respective troops crashed on Earth. The smartest, strongest, and most intelligent Decepticons left Cybertron to colonize nearby galaxies, leaving a token leadership of overly cruel and duplicitous Decepticons to control Cybertron. These Decepticons who left their homeworld became "The Second Generation" Decepticons, who were led by the villainous Liege Maximo. On the final page of the last issue, the Liege Maximo indicates that the Decepticons and all their descendants came from him, which indicates that he is the original founder of the Decepticons.

Animated series
The Decepticons are a malevolent race of robot warriors, brutal and merciless. The Decepticons are driven by a single undeviating goal: total domination of the universe. In the war that raged between the Decepticons and Autobots for millions of years, their home planet of Cybertron was drained of its once rich sources of energy. In hopes of finding new reserves for making Energon, their basic fuel, the Decepticons followed the Autobots to Earth. Under the leadership of the ruthless Megatron, they continued their efforts to destroy the remaining Autobot forces. They attacked throughout the Earth and in space, using their underwater Decepticon headquarters as their staging base and bunker. But, by the year 2000, the Decepticons had proven to be the losers of the last Great War. In an attempt to bolster the Decepticons' strength, the evil planet gobbling world, Unicron, reformed Megatron into a new robot named Galvatron to lead the Decepticons in a new era. Now, in 2006, the Decepticons have retreated to a burned out hulk of a planet called Chaar, a world of ash and ruin. Galvatron and his Decepticon lieutenants, the sleek and awesome Cyclonus, and the mighty Scourge, the master of the dreaded Sweeps, strike fear throughout the universe. Their ultimate goal: to reconquer Cybertron and destroy the Autobots.

In the original animated series continuity, the Decepticons owe their warlike ways to their faction's origin as military hardware robots, created by the five-faced aliens, the Quintessons, while the Autobots were designed as consumer goods. Following the rebellion that forced the Quintessons off the planet, the Decepticons - as they named themselves - lusting for power, began a civil war. The Autobots could not hope to match the superior firepower and battlefield powers of the Decepticons, and instead turned to stealth, developing the art of transformation, modifying their bodies so that they could assume other forms. With this additional power, the Autobots were able to win the conflict, and a period of peace began, known as the Golden Age of Cybertron, when energy was plentiful and the planet shone with a golden hue.

Working in secret, the Decepticons also developed transformation technology, along with new robot-mode flight powers, and, under the command of one of the first of this new breed, Megatron, they attacked one of the capital's outer cities, killing the current Autobot leader. As the knowledge of this act was not made public, younger robots continued to idolize these powerful new flying robots - until one such young robot, Orion Pax, was duped by Megatron into allowing him access to an energy storage facility. Megatron turned on Pax and fatally injured him, but the ancient Autobot, Alpha Trion, then rebuilt Pax into the first of a new, battle-hardy breed of Autobot - now, he was Optimus Prime, leader of the Autobots, who led them against Megatron and the Decepticons as the civil war erupted once more.

After five million years, Cybertron was nearly depleted of all its energy when the Decepticons received word that the Autobots were planning to search for new energy resources. Following the Autobot ship, the Ark, in their ship, the Nemesis, the Decepticons boarded the Ark and engaged the Autobots on their ship. Unfortunately, the Ark drifted too close to Earth and crashed landed in a volcano. All the occupants of the Ark went into emergency stasis, while the Decepticons on the Nemesis ejected to South America. There, they scanned the forms of local insects, becoming the Insecticons. After 4 million years, the volcano the Ark was wedged in, erupted in Earth year 1984, restarting its computer, Teletraan I. Sending out Spy Satellites, Teletraan created alternative forms out of Earth vehicles for any transformers caught in its scanning beam (like F-15 Eagles for Skywarp and a Semi-trailer truck for Optimus Prime). The Decepticons awakened first but later the Autobots did too, thanks to Starscream's carelessness.

By the Earth year 2005, however, the Decepticons had succeeded in completely conquering Cybertron. In a final attempt to destroy the remaining Autobots they attacked Autobot City on Earth but were defeated. On their way back to Cybertron Astrotrain encountered difficulty leaving Earth's gravity due to an over-large payload. Starscream convinced the others to jettison all the damaged and dying decepticons, including Megatron. Megatron was then found by Unicron who offered to rebuild him in return for destroying the Autobot Matrix of Leadership. Megatron was reformed into Galvatron who returned to Cybertron, destroyed Starscream for his treachery and reassumed command of the Decepticons. Galvatron then attempted to use the Matrix against Unicron, provoking him into attacking Cybertron. The attack coupled with the temporary loss of Galvatron shattered the decepticon forces, allowing the Autobots to reclaim their home planet. The Decepticons fled to the planet Chaar, a world of ash and ruin, where they struggled to survive until Galvatron returned to them and reorganized them into a fighting force, battling to take back Cybertron.

Dreamwave Productions
After Autobot leader Optimus Prime and Decepticon leader Megatron disappeared in a Space Bridge accident several million years ago, the Autobots and Decepticons split up into several factions. One of those who broke away was Ratbat, who quickly took the opportunity to form his own power base. Gathering loyalist followers and setting up their HQ in the Polyhex region of Cybertron the Ultracons quickly came into conflict with the Autobot splinter faction The Wreckers. However, Ratbat added a secret weapon to his ranks - the combiner team the Constructicons, in flagrant defiance of treaties banning their use in the civil war as both Autobots and Decepticons splintered.

When the Ultracons attacked the Tagan Heights in order to seize its factories for themselves, they were opposed by the Wreckers. Ratbat quickly unleashed the Constructicons in their combined form of Devastator, routing the Wreckers. However, Ratbat was prevented from executing his rival Springer by the arrival of the Protectobots in their combined form of Defensor. The Autobot combiners were hoping to prevent an arms race between the various factions by stopping combiners signing up with the factions. After the battle devastated the very manufacturing facilities the two factions hoped to acquire, the Protectobots separated and defeated Devastator via sabotage. With their most potent weapon gone, the Ultracons retreated.

Later, the Ultracons joined with the Decepticons under Shockwave to support a peace accord with the Autobots, fighting off Starscream's Predacon faction. However, Megatron's return with an army of Seeker clones would bring a halt to the peace proceedings, and the Ultracons would be reabsorbed into the Decepticons.

IDW Publishing
The origins of the Decepticons in the IDW continuity would be explored in The Transformers: Megatron Origin, which confirmed Megatron as their founder in this timeline. A lowly miner, Megatron penned a manifesto against Functionalism, the forcible assignment of cybertronians to jobs based on their vehicle modes, which was in force at the time on religious grounds. Megatron began to work against the practice through peaceful protest, but following imprisonment and a brutal beating by police, he determines that force is the only way to bring about equality. Ironically, on his way out the station after having been saved from a government ordered 'accidental-death-by-interrogation', he first meets the future Optimus Prime, who would go on to use his original writings as the basis for the Autobot movement.

On the run after an unrelated attack on an Autobot Senator after a mining dispute, Megatron stumbled onto underground gladiator games in the state of Kaon and quickly became one of its rising stars. However, it only truly became more than this when Ratbat, then a senator seeing an opportunity for profit, dispatched Soundwave to arm the gladiators. Megatron's forces then began to commit acts of terrorism increasing in scale and daring, attracting the attention of Sentinel Prime. Kidnapping Senator Decimus as a symbol (with the help of new airborne recruit Starscream) Megatron issued a rallying call to the gladiators and their supporters, promising to unite them under one badge and make the whole planet their arena. Sentinel Prime's forces then found and arrested them en masse. This was all part of Megatron's plan: a supposedly informant Starscream massacred the Autobot Senate and freed the prisoners, signalling revolution and anarchy in Kaon. Megatron eventually seemed to have killed Sentinel Prime one-on-one, and the Decepticons were forged.

In this continuity, there are a number of differences to the traditional norm. The Decepticons are no longer based on Cybertron due to its devastation by rogue scientist Thunderwing. Instead, they have spread out through the galaxy, using their transforming abilities to infiltrate target worlds (such as Earth or Nebulos) disguised as local vehicles and equipment. They then use their modes to create conflict, leading to complete anarchy as the world's political and social systems collapse. The Decepticons then move in and harvest the world's raw resources. How the world is pacified is unknown, although The Transformers: Spotlight issue on Sixshot revealed he is often sent in to complete the destruction of entire worlds. As in other continuities, Megatron is in command, and thus far has had to deal with the rebellion of Starscream's Earth infiltration unit and the reemergence of Thunderwing, as well as moving on to Earth in order to harvest the extremely powerful Energon variant Ore-13, something which promises to give the Decepticons a significant edge over the Autobots.

The IDW continuity is unique in that it portrays the Decepticons as originally a force for good, standing up against oppression, who eventually themselves became oppressors. A large number of those who would become Autobots, including Optimus Prime himself, even support the Decepticon cause in the beginning, before it turns violent.

Beast Era
The Decepticons eventually evolve into the Predacons, a faction in the fictional Beast Wars universe created by Hasbro. Previously the name Predacon had been associated with the Transformer group which combined to form Predaking. The faction made their first appearance in the Beast Wars toyline and animated series, and have been represented in various Transformers animated series, comic books and toylines since.

A Predacon is most commonly represented as a Decepticon descendant with a new smaller, more energy-efficient body. This body transforms into an animal instead of a vehicle. Though the Decepticons and Predacons share many similarities, several important differences exist, such as leadership and ideals (for example, choosing a pretense of peace with the Maximals, rather than maintaining a war at all costs). It is not completely known as to how the faction gained predominance over all the other various Decepticon factions, although the comic book miniseries Beast Wars: The Gathering has provided hints that their heritage may be found in the original combiner team Predacons, hints confirmed by the subsequent Timelines: Dawn of Futures Past, which confirmed that Razorclaw, Divebomb and the other Predacons had been working behind the scenes.

Transformers: Robots in Disguise
In Transformers: Robots in Disguise, the Decepticons were not the name of the evil faction, but rather a sub-faction created by Megatron. The main faction was instead the Predacons (carrying the name of the main villains over from Beast Wars), who transformed into mechanical animals. Realising that their ability to travel as Earth vehicles gave the Autobots an edge in stealth, Megatron decided to take drastic action.

Megatron recovered the protoforms of six Autobots that had crashed to Earth some time before looking for Fortress Maximus. Scanning them with vehicle modes at a military base and infusing them with a fraction of his spark energy to ensure their new loyalty, the first five became the Commandos. But when Megatron tried to scan a tanker truck as an alternate mode for the leader Scourge, Optimus Prime was scanned as well, resulting in much of Prime's personality being scanned as well. With an infusion of Megatron's spark energy to complete the concoction, Scourge was born, emerging from his pod as a dark twin of Optimus Prime, who proved that Megatron's programming had over-ridden any Autobot goodness within him. Appointed leader of the new "Decepticons", they quickly established themselves as a force to be reckoned with.

The newly created Decepticons would try many schemes over the course of the series, including attacking various dams and faking their defection to the Autobots. But their leader, Scourge, had grander ambitions in mind. The copy of Optimus' personality had been warped by Megatron's evil influence, but the individuality of Optimus remained and Scourge now wanted to control both Predacons and Decepticons. He would seize his chance when Fortress Maximus was awakened, directing him to destroy Galvatron (Megatron's new form). The attempt failed and Galvatron took a terrible revenge - he stripped the Decepticons of their individuality, turning them into little more than drones. After Galvatron's defeat by Omega Prime, the Decepticons were among those sent back to Cybertron for imprisonment, except for Sky-Byte.

Unicron Trilogy
The Decepticons appeared in Transformers: Armada, Transformers: Energon, and Transformers: Cybertron as mostly jets and military vehicles.

In the Unicron Trilogy continuity, the early history of the war is not expounded upon. The Decepticons, however, remain the main antagonists, with whom the Autobots have been warring for quite some time. The enslavement of the small, power-enhancing Mini-Cons has long been a major goal of the Decepticons, and the Mini-Cons are seen as key to winning the war.

Near the end of Armada, the Autobots and Decepticons join forces to defeat the planet-devouring Unicron, and the beginning of Energon sees them as uneasy allies. Many return to Megatron's side when he returns.

Cybertron sees a much smaller Decepticon force than the previous two series, and what has become of most of the survivors of the earlier series remains unknown. As Megatron's true plans are discovered, most of the Decepticons, unwilling to be sacrificed as pawns as their leader seeks to remake the universe, again battle a common foe alongside the Autobots. The end of the series sees three of the comic-relief characters, with a reluctant Thundercracker, desiring to begin a new Decepticon force, and take off in a makeshift rocket. It makes it as far as Mars before it crashes, as the ending credits montage of the final episode shows. The foursome survives, however. The fates of the other surviving Decepticons remains unknown, though evidence of Starscream's survival is discovered.

Live-action films
The Movie Prequel comic by IDW Publishing would reveal their origins. Originally Optimus Prime and Megatron ruled Cybertron together, due to the power of the Allspark, a mysterious device that could give life to Transformers - and keep Cybertron itself alive. Eventually, Megatron began to desire its power for himself and assembled a like-minded army of separatists - the Decepticons. Megatron soon struck, and Cybertron became embroiled in civil war. However, this would later seem to be changed by the sequel films, where it is revealed that Sentinel Prime was the first leader of the Autobots and The Fallen the founder and supreme master of the Decepticons. Optimus would only take command after Sentinel was seemingly killed in action and Megatron served as leader of the second incarnation of the Decepticons with The Fallen as his master and mentor due to the latter's incapacitation. The subsequent prequel novel Transformers: Ghosts of Yesterday revealed that Starscream seized command of the Decepticons after Megatron's disappearance.

Transformers (2007 film)
The Decepticons make an appearance in the 2007 the Transformers live-action film. They include Megatron (Hugo Weaving), Starscream, Frenzy, Blackout, Barricade, Scorponok, Brawl and Bonecrusher. Like the Autobots, the Decepticons are designed to look more organic in the movie. While the Autobots have more human-like appearances, the Decepticons are more monstrous; they tend to resemble carnivorous insects, mammals and birds. In the film, the Autobots intend to use the AllSpark, the object that created their robotic race, in an attempt to rebuild Cybertron and end the war, while Megatron and the Decepticons desire to control it with the intention of building an army by giving life to the machines of Earth. Ultimately, nearly all the Decepticons who participated in the first battle (except Starscream, Scorponok and Barricade) are killed and the AllSpark is used by Sam to kill Megatron, thus making the Autobots accept Earth as their new home, while allying with humans to continue to hunt down remaining Decepticons.

Transformers: Revenge of the Fallen (2009 film)
In Transformers: Revenge of the Fallen it is revealed that The Fallen, a Prime who turned to evil, is the founder of the Decepticons, with Megatron being the leader of its second incarnation, accepting The Fallen as his master and mentor. This would seem to imply the Decepticons were originally Autobots who turned evil. In the final battle in the second movie, there are many Decepticon casualties including Devastator (destroyed by a rail gun), Mixmaster (cut in half and his head stomped off by Jetfire), Rampage (killed by Bumblebee), Ravage (stripped from his spine by Bumblebee), Scrapper and Long Haul (killed by an airstrike), Scorponok (head crushed by Jetfire) and the Fallen himself (spark ripped out by Optimus Prime). Also killed in the movie are Sideways (while still in his vehicle mode) by Sideswipe and Demolishor, killed at the beginning by Optimus and Ironhide. Following the Fallen's death, Megatron officially takes full command of the Decepticons at this point, swearing vengeance.

Transformers: Dark of the Moon (2011 film)
The Decepticons return in Transformers: Dark of the Moon with the plan to use a Space Bridge to bring an army to Earth and transport Cybertron into orbit to rebuild the planet with Earth's natural resources and human population as slaves. Megatron and Sentinel Prime planned this a long time ago, planning to meet on Earth before Megatron went after the Allspark and ended up frozen and the Ark was shot down by Decepticon fighters (few Decepticons knew of the plan) and crashed on Earth's Moon in 1961. This was detected by the Americans and the Russians and the space programs were created to reach the ship and study it. However, by 1963 the Decepticons had raided the Ark and removed all but five of the pillars and hid them on the Moon along with an army of hundreds of Decepticons, awaiting the time that the pillars would be used. After humans successfully reached the Moon and explored the ship, Soundwave and Laserbeak approached various humans including Dylan Gould's father to get them to shut down the American and Russian space programs.

The Decepticons established a network of human collaborators who kept the humans from noticing the Decepticons actions on the Moon, later led by Dylan Gould. After a human collaborator named Alexi Voshkod leads NEST to Chernobyl where they retrieve an engine part from the Ark, the Decepticons reemerge for the first time since Egypt in the form of Shockwave and his Driller who attack, but retreat. This leads the Autobots to retrieve Sentinel and the remaining space bridge pillars and revive him and as this is going according to plan, Megatron orders Laserbeak to kill all the human collaborators but Dylan Gould which he does. Before he's killed, one of the collaborators named Jerry Wang alerts Sam Witwicky to this and he alerts NEST which ignores it. Sam and Seymour Simmons find out about the Decepticons plot from pictures a Russian probe got of the Decepticons hiding the pillars and the Dreads are sent to attack the convoy protecting Sentinel, but are killed by Bumblebee, Dino, Sideswipe and Ironhide.

After Sentinel reveals his true colors and murders Ironhide, he sets up the pillars in the National Mall and brings the Decepticon army consisting of hundreds of Decepticon soldiers and Decepticon ships to Earth. With the army in hand, the Decepticons promise to bring no war against the human population in exchange that the Autobots must leave Earth. While taking off into the atmosphere, Starscream destroyed the ship, knowing that mankind will resist on their own. Some of the Decepticons scatter around the world to set up the pillars, but most of the army invades Chicago, devastating the human population and taking over. The Autobots, NEST and a team of mercenaries led by Sam Witwicky and Robert Epps invade Chicago and battle the Decepticon army, aided by the United States Army, and U.S. Air Force. The group manages to take out much of the army including Laserbeak (killed by Sam Witwicky and Bumblebee), the Driller (killed by Optimus), Starscream (killed by Sam Witwicky and Sergent Lennox), Soundwave (killed by Bumblebee), Barricade (killed by NEST) and Shockwave (killed by Optimus) and disrupt the Control Pillar, forcing Sentinel into battle. The combined human and Autobot forces overwhelm Sentinel and his remaining forces and Tomahawk missiles destroy most of the Decepticon fighters, forcing Sentinel to flee before engaging Optimus one-on-one. Dylan Gould reactivates the space bridge, but Sam Witwicky kills Gould, and Ratchet and Bumblebee destroy the Control Pillar, causing Cybertron to collapse and ruining the Decepticon plan. Also, the remaining Decepticon ships are sucked up into Cybertron by the space bridge and are destroyed with the planet.

Sentinel nearly kills Optimus, but Megatron, convinced by Carly Spencer that he would end up working for Sentinel unless he could get rid of him, turns on Sentinel and severely injures him. Upon seeing Cybertron being destroyed with the space bridge, Megatron attempts to make a false truce with Optimus, but Optimus doesn't fall for it and kills Megatron and Sentinel. While the loss of human life in Chicago is massive, the battle is a major victory for the Autobots as they eliminated most of the Decepticon army on Earth (including Gould by traitor) despite there being some still scattered around the globe and the entire Decepticon command structure was destroyed with all of its members killed in the battle.

Transformers: Age of Extinction (2014 film)
While the regular Decepticons do not actually appear in Transformers: Age of Extinction (except Megatron who was reborn as Galvatron), they do play a role in the story as secondary antagonists. The "Battle for Chicago" (climactic battle of Dark of the Moon) has become a 9/11 like event to the public 5 years after the third film and a ruthless government organization named Cemetery Wind have begun hunting Decepticons who are still scattered around the globe and secretly hunted Autobots alike when the alliance between humans and Autobots ended (including alliance between humans and Decepticons as Dylan Gould is already dead). It was mentioned that there might still be less than a dozen Decepticons and Autobots hiding on the planet Earth by Harold Attinger (Kelsey Grammer). The film's antagonist is instead the neutral assassin and bounty hunter Lockdown, who may have been a Decepticon at one point, sent to Earth to hunt Optimus Prime by the Transformers' creator race (unseen in the movie). Lockdown and Attinger make a three-way deal along with a company named K.S.I., Kinetic Solutions Incorporated, by the boss, Joshua Joyce, who uses the original Decepticons' (and Autobots') remains to build new Transformer prototypes which can be controlled by humans and must eliminate those who cross their path such as the Yeager family who know this secret.

Their crown jewel is Galvatron, who becomes possessed by Megatron's (Frank Welker) consciousness and then possesses the other prototypes and turns them into Decepticons controlled by him. Galvatron plans to use the Seed, a device that can create Transformium, the metal Transformers are made of, to rebuild his Decepticon army and conquer Earth, when Joyce was called by Cade Yeager, and, knowing that he was manipulated by Attinger and Galvatron, escapes with the Seed. After the final battle in Hong Kong, all the Decepticons are destroyed by Cade, the Autobots and Dinobots, except Galvatron, who escapes after Lockdown and Attinger are killed by Optimus Prime, vowing to return one day, for he is reborn, before Cemetery Wind dissolved by their crimes and branded as a terrorist organization.

Barricade, Soundwave and a number of other Decepticon "protoform drones" appear in archival footage from Dark of the Moon presented by Attinger. Various deceased Autobots and Decepticons appear throughout the film in Chicago and in the KSI lab, including Megatron's head.

Transformers: The Last Knight (2017 film)
The Decepticons return in Transformers: The Last Knight with the revived leader, Megatron, when a war for survival has commenced between the human race and the Transformers and set up a military organization of TRF (Transformers Reaction Force). The Decepticons began to search for the staff when Barricade reports to Megatron that human has found the mysterious talisman they're seeking and suggests using TRF their purposes. Megatron contacting TRF abducting two CIA agents and offered an exchange to release his group of Decepticons. During the final battle, they assist the sorceress Quintessa in draining the life-energy from Earth to repair a damaged Cybertron and began to defend the from the Autobots and military. But the Autobots broke through and most of the Decepticons began to die and were defeated.

Bumblebee (2018 film)
The Decepticons return in Bumblebee, where they are first seen during the final battle for Cybertron against the Autobots. The Decepticons on Cybertron are co-led by Shockwave and Soundwave, who proceed to force the Autobots off of Cybertron.

At some point prior to this battle, the Decepticon Seeker Blitzwing was sent to Earth by the Decepticons for unknown reasons. After the Autobot scout B-127 lands on Earth, Blitzwing intervenes in an ensuing skirmish between the Autobot and the human special operatives group known as Sector Seven. He proceeds to engage B-127 and interrogate him about Optimus Prime's whereabouts, even ripping out the scout's vocal processor and attempting to execute him. Before he could do this, however, B-127 attached one of Blitzwing's missiles onto Blitzwing's body, causing him to explode and die in the process.

Decepticon Triple-Changer agents, Shatter and Dropkick are sent to intercept lieutenant Cliffjumper, one of the Autobots that escaped at the beginning of the film, in the hopes that they may find the Autobot leader, Optimus Prime. Cliffjumper refuses to reveal Optimus' whereabouts and thus subsequently fatally sliced in half by Dropkick.

Transformers Animated
In Transformers Animated, the Decepticons are usually bigger and stronger than the Autobots and usually turn into jets and military vehicles. It takes a group of Autobots to defeat one Decepticon and almost all Decepticons have red eyes and can fly[citation needed]. Most Decepticons in the series are similar to the original G1 series. In the Animated Series Blitzwing, for example, retains his triple changer modes of a tank and jet and also retains his G1 tan and purple color scheme.

This series depicts a Decepticon force that is far from what it apparently once was, with Megatron having been chasing the Allspark for millions of years prior to the series' beginning. The Decepticons have been "wandering the periphery" for some time, but have been driven off of Cybertron long ago. These Decepticons see themselves as "freedom fighters", seeking to throw off "Autobot tyranny", and their plans tend to revolve around returning to Cybertron with the means to retake it. What started the great war in this series was some of the Cybertronians being kicked off of Cybertron for being freaks, eventually forming an alliance called the "Decepticons". Now the Decepticons fight for their rights and try to take back what was rightfully theirs.

Transformers: Timelines
A series of stories printed by Fun Publications, set in a variety of continuities, the Decepticons of alternate worlds can vary greatly from the classics Decepticon image. In the Transtech storyline, the Autobots and Decepticons are merely political factions on a Cybertron which never broke out into war. Lines between good and evil are more blurred.

In the Shattered Glass storyline, a group of Decepticons, led by Megatron, are the mirror images of the personalities of most Decepticons. In this world the Decepticon logo is red, and worn by heroic Transformers who seek to protect the weak and oppressed from the evil Autobots.

Aligned Continuity
The main group of Decepticons, led by Megatron, in the 2010 computer animated series Transformers: Prime is much larger than in most continuities considering their ranks are bolstered by a clone army of drone named Vehicons (a possible homage to Beast Machines) who all look-alike though some have different vehicle modes and have personalities. The Vehicons are sometimes the center of a joke often shown unable to do their job and are very incompetent as well as constantly being killed off way too easily by the Autobots. However The Decepticons don't appear in the series Transformers: Rescue Bots, though it has been heavily implied that the two shows take place in the same continuity. The Decepticons are briefly mentioned by Heatwave in the episode "Bot to the Future", (who also mentioned Unicron in an earlier episode) when the Autobots arrive in an alternate timeline ruled by Morbots (who resemble Vehicons), Bumblebee believes the Morbots are Decepticons, only for Heatwave to correct him. In a later episode, the recruits retrieve what appears to be a cassette tape from a derelict spaceship and it transforms into an injured Laserbeak. The group disagree over whether to help him as he is a Decepticon (Hotshot in particular is against doing so) and it's ultimately revealed that Wedge was originally built as a Decepticon but was given the chance to instead be an Autobot (a fact he kept from the others due to the prejudices shown in this episode). In the end, they allow Laserbeak to stay at the academy while he recovers.

Transformers: War for Cybertron
In Transformers: War for Cybertron and Transformers: Fall of Cybertron, Megatron has come across a legendary element called "Dark Energon". He infiltrates Starscream's space station in order to corrupt himself with it. He then kills the Autobots' leader Zeta Prime and obtains the Omega Key, which then activates Omega Supreme. After Megatron defeats Omega Supreme, he goes to Cybertron's core and corrupts it with Dark Energon. Optimus Prime becomes the new Autobot leader, and heads into the planet to rescue Omega Supreme. He opens the Omega Lock into the core after Ratchet purges the Dark Energon corruption. Optimus discovers that he is too late to save Cybertron from the Dark Energon corruption, and the core must shut down for about one million years to survive. Optimus is giving a piece of Cybertron's spark for safe-keeping (The Matrix of Leadership). When Optimus evacuates the Autobot forces from Cybertron, Megatron tries to stop them from leaving. Silverbolt, Air Raid, and Jetfire try to stop it, but the space station transforms into Trypticon and crashes on Cybertron. Optimus and his Autobots then defeat Trypticon. Megatron has Trypticon rebuilt into the Nemesis. The Nemesis pursues the Ark, and then Megatron shoots Bumblebee. They go into a space bridge, and leave Cybertron to a lifeless wasteland.

Transformers: Prime

This article's plot summary may be too long or excessively detailed. Please help improve it by removing unnecessary details and making it more concise. (December 2019) (Learn how and when to remove this template message)
By the time of Transformers: Prime, it has been three years since the Decepticons last attacked Earth and the Autobots still await their return. After the death of Cliffjumper, the Autobots fight to protect the Earth from the Decepticons and befriend three young humans while preparing for the return of Megatron, who has been missing for three Earth years. Megatron plans to use Dark Energon to raise an undead army of Cybertronians and defeat the Autobots. The Autobots destroy his space bridge and Megatron is believed killed in the process. Starscream becomes the new Decepticon leader. After Megatron's apparent death, new Decepticon leader Starscream does not stray from Megatron's path and recruits several new Decepticons. Starscream attempts many missions to destroy the Autobots and find their headquarters, while keeping a shard of the "last" remaining Dark Energon that he took out of Megatron's chest. In the episode "Out of His Head", Megatron returns after an incident where he takes control of Bumblebee's mind. Megatron then reclaims leadership of the Decepticons, keeping a strict eye on Starscream. After Starscream uses Megatron's share of Dark Energon, he tries to find more to once again bring back his "un-dead army". At the end of the episode "Partners", Starscream became an independent, striking out on his own, having become fed up with being disrespected. During the final four episodes of the season, the Autobots unwillingly team up with Megatron to battle a legendary threat to Earth's existence, Unicron.[3] To defeat Unicron, Optimus uses the Matrix of Leadership.

With this sacrifice, he not only loses the Wisdom of the Primes but also his memories of the war and his comrades, and joins Megatron as a Decepticon under his old name, Orion Pax. Orion, under the false truths about the Autobots being evil and the Decepticons being good, Megatron, then sets about decoding the Iacon Database. The Autobots save him, journeying to Cybertron and back to recharge the Matrix of Leadership and help him regain his memories, but the Decepticons continued to search for the relics Orion did decode throughout Season 2, managing to decode all of them thanks to Soundwave's persistence and computer skills. Starscream meanwhile attempts several times to create personal armies and an alliance with MECH, only for the human terrorists to take his T-cog to construct Nemesis Prime as well as a failed attempt to assassinate Megatron. The Insecticons also join the crew of the Nemesis when Airachnid tries to eliminate Megatron for trying to terminate her previously. Airachnid is later imprisoned in a Stasis pod by Acree and held as a prisoner in the Autobot Base. Team Prime also is granted another Autobot when Smokescreen, the rookie warrior and bodyguard of Optimus Prime's mentor Alpha Trion, arrives on Earth in a Decpticon escape pod and Megatron gains MECH leader Silas, who has taken Breakdown's body for his own after nearly being killed by Optimus Prime, as an ally until he fails Megatron's expectations, causing Cylas (as he calls himself) to end up as Knockout's science experiment. When Optimus claims possession of the Star Saber and learns of the last four Iacon relics: the Omega keys (devices that can restore Cybertron), Megatron replaces his arm with an Ancient Prime's arm, so he can use the Forge of Solus Prime to turn a chunk of Dark Energon into the Dark Star Saber. In the battle that follows, Megatron destroys the Star Saber, robbing Optimus of Victory.

Defeated, Optimus Prime retreats, gaining the first Omega Key in the process due to Smokescreen's clever thinking. Knock Out obtains the second however using the Resonance Blaster, and Starscream obtains the third using a dose of speed-enhancing Red Energon. The fourth and final one is revealed by Soundwave to have been inside Smokescreen, placed there to make sure it got to Optimus from Alpha Trion. After the Decepticons kidnap him, Knock Out uses the phase shifter to remove it. Megatron also finds out the purpose of the keys, by using the cortical psychic patch on Smokescreen. When Megatron leaves Knockout alone with him, Smokescreen and Knockout fight over the phase shifter, ending with the Autobot placing Knockout in a wall and leaving him there. He takes the one the Decepticons had and his and escaped the ship, starting a free fall battle, resulting in Smokescreen's escape. Starscream then goes on to steal the three keys the Autobots had. Starscream, now with all 4, uses them as a peace offering to rejoin the Decepticons.

Dreadwing becomes angry that Megatron allowed Starscream to rejoin the Decepticons after seeing that Starscream turned Skyquake into a Terrorcon. He gives Optimus the Forge of Solus Prime, and attempts to kill Starscream but is killed in the process by Megatron. Knockout and Starscream use the keys and find out the location of the Omega Lock, with Optimus already knowing its location and turning the Goundbridge into a spacebridge, and reforging the Star saber. The Autobots head to Cybertron, fighting off the Decepticons with their Iacon relics, take back the keys and manage to kill all the Vehicons, managing to reach the Omega Lock. Sadly, the victory is short lived, as the Decepticons force them to give them back the keys, or they will expose Jack, Miko, and Raf to Cybertron's toxic atmosphere. The Autobots are forced to yield and give them the keys, which Megatron uses to rebuild the Hall of Records. He then opens a space bridge, and tries to terraform Earth into a new planet he considers calling New Kaon.

Optimus, fearing for humanity and unwilling to let another world be destroyed, takes his Star Saber and cuts off Megatron's arm and destroys the Omega Lock, leaving Cybertron in its liveless state forever. The Autobots retreat back to Earth, only to find that Jasper, Nevada has now been changed into a giant fortress. The Nemesis lands there, and Optimus discovers the Decepticons have discovered the location of their base. Under Megatron's orders, Starscream leads an armada of Vehicons and Insecticons to start attacking the base. Wheeljack and Agent Fowler try and hold off the new Vehicons as long as they are able, trying to allow Team Prime to escape. Optimus has Team Prime split up to different parts of the country in the hope of making them more difficult to hunt down, with Optimus remaining behind to ensure the Decepticons could not follow. Outside, Wheeljack's ship, the Jack Hammer is shot down by Starscream. The Decepticons then blow up the base with Optimus inside as he destroys the Ground Bridge with Agent Fowler and June Darby watching in horror as the base explodes. Megatron and Starscream then fly down to the remains of the base to search for survivors. Celebrating their victory, they fail to notice Optimus's arm sticking out of the rubble.

In Season 3, the Decepticons hold earth hostage with their fortress, christened as Darkmount, and search for the autobots across the United States, To track the autobots, the recently returned Shockwave brings with him a Predacon to hunt down the scattered Autobots. When the Predacon fails, the search continues under Starscream's command. This search is put on hold however when the Autobots unite under Ultra Magnus to assault Darkmount. However, as the assault continues, it is the Decepticons who emerge victorious even with the loss of the Predacon, capturing Team Prime. However, Optimus Prime, returned to health by Smokescreen's use of the Forge of Solus Prime, attacks Darkmount, defeating Megatron in one on one combat in the air and destroying the fortress' fusion cannons so that the military can destroy the fortress. Forced to use the Nemesis as their base once again, Megatron begins sending units to hunt down Predacon fossils in the hope of gaining an advantage over the Autobots through cloning an army. The return of the Predacon (Predaking himself) adds to this advantage. However, the Decepticon ranks are thinned when Starscream and Knock Out accidentally turn Cylas into a Terrorcon that can turn other Decepticons into Energon vampires through a mixture of Synthetic and Dark Energon. This plague also leads to Airachnid's escape and her retaking control of the Insecticons, forcing Soundwave to teleport her armada to one of Cybertron's moon's and leaving them to be eaten by her as she has become a Terrorcon hybrid. This accident forces the cloning process to be sped up on Project Predacon due to the loss of more than half of Megatron's forces. However, Project Predacon is terminated when Predaking reveals he is capable of higher thought as well as transformation, forcing Megatron to lure the Autobots into destroying the cloning laboratory for him. With the destruction of the lab, it is revealed that Predacon CNA combined with Synthetic Energon is capable of producing cybermatter. Attention is then focused on the restoration of the Omega Lock and the completion of the Synthetic Energon formula. To complete it, Ratchet is kidnapped and forced to aid in completing the formula, at which he succeeds. Megatron however does not count on Ratchet turning Predaking against him, nor the Autobots assaulting the Nemesis. With this, Megatron rallies his troops for a final stand. Though he kills Bumblebee and nearly kills Optimus, Megatron is killed by a resurrected Bumblebee who emerges from the Omega Lock and stabs him in the spark with the Star Saber, sending him tumbling to the earth far below. With Megatron's death, and Soundwave trapped in the Shadowzone by Jack and Miko, the remaining Decepticons are killed, imprisoned or retreat with Shockwave and Starscream to the ship's escape pods.

In the film "Predacons Rising" - which serves as the series finale - it is further revealed that while many Decepticons were imprisoned, some defected to the Autobots to help rebuild Cybertron. Starscream and Shockwave, however, remain at large and having cloned two new Predacons they have named Skylynx and Darksteel, they continue to hunt Predacon bones to clone more soldiers. When Megatron returns, Starscream is at first overjoyed to have their leader back, but upon realization that Unicron is overshadowing and controlling him, Starscream retreats while Shockwave and the Predacons fight. Shockwave is also the one who suggests that the Predacons, now under Predaking's full leadership, help the Autobots defeat Unicron and his undead army. Starscream and the Decepticon prisoners aboard the Nemesis attempt to take control of the ship, only for Knock Out to change allegiance to the Autobots and imprison Starscream while the remaining mutineers are killed. Though Starscream escapes, Knock Out stays to help stop Unicron. When Unicron is defeated and imprisoned by Optimus Prime, Megatron is freed from his control and officially disbands the Decepticons and flies off to start a new life, as he is tired of conquest and no longer wishes to oppress anyone, having just experienced it himself. Starscream takes Megatron's throne but his rule is short-lived when the Predacons arrive to settle old scores, the outcome of which remains unknown.

Transformers: Robots in Disguise
Five years after the events of "Predacons Rising", Bumblebee is alerted to the existence of a new group of Decepticons on Earth. He eventually assembles a team-consisting of Autobots Strongarm, Sideswipe, Grimlock, and Fixit-to contend with these new Decepticons, which include the likes of Underbite and Bisk. Steeljaw, in particular, is the main antagonist, leading a group of Decepticons consisting of bounty hunter Fracture, crime boss Thunderhoof, the metal-consuming Underbite, and the crab-like Clampdown, in hopes of giving the Decepticons their own world. Towards the end of season one, Megatronus/The Fallen, the first Decepticon, contacts Steeljaw and convinces him to build him a space bridge so he can return to Earth. Steeljaw does so, but once on Earth, Megatronus attempts to destroy both Earth and Cybertron. Steeljaw tries to stop him, but Megatronus uses his telepathic powers to throw him into the distance. Ultimately, Megatronus is killed by Bumblebee's team and the rest of Steeljaw's pack is imprisoned.

In season two, Steeljaw comes across a group of Decepticons who have turned the Alchemor into their own base, enslaving the Mini-Cons there as well. These Decepticons are led by the strict Glowstrike, ex-pirate Sabrehorn, and the gruff Scorponok. Steeljaw manages to reform his pack by using Scorponok as a distraction while he infiltrates the Autobots' scrapyard and frees Clampdown and Thunderhoof from stasis, along with the insane Springload, the revolutionary Quillfire, hyperactive Bisk, former gladiator Groundpounder, and spy/saboteur Overload. Scorponok is arrested by Bumblebee's team, with Steeljaw fabricating a story that Scorponok had betrayed them. At the end of the season, Steeljaw usurps Sabrehorn and Glowstrike and becomes leader of the escaped Decepticons (collectively known as "Decepticon Island"), but Bumblebee's team sets off a specialized bomb that places all Decepticons on board the Alchemor in stasis, including Steeljaw. Optimus Prime, Windblade, Ratchet, and the former Mini-Con prisoners then take the recaptured Decepticons back to Cybertron while Bumblebee decides to establish a permanent Autobot base on Earth."""

In [ ]:
additional_data = user.dataloader.dataset.tokenizer(text)
len(additional_data["input_ids"])

In [ ]:
seq_length = 512
B = len(additional_data["input_ids"]) // seq_length
print(B)
data = torch.as_tensor(additional_data["input_ids"][:seq_length * B]).reshape(B, seq_length)
user.data_key = "input_ids"
input_data = dict(input_ids=data, labels=data)
true_user_data = dict(data=input_data, labels=input_data)
server_payload["metadata"].shape = [seq_length]
server.secrets["ImprintBlock"]["data_shape"] = [seq_length]
user.num_data_points = B

In [ ]:
shared_data, true_user_data = user.compute_local_updates(server_payload, custom_data=input_data)
shared_data["metadata"]["num_data_points"] = B

### Reconstruct user data:

Now we launch the attack, reconstructing user data based on only the `server_payload` and the `shared_data`. 

For this attack, we also share secret information from the malicious server with the attack (`server.secrets`), which here is the location and structure of the imprint block.

In [ ]:
reconstructed_user_data, stats = attacker.reconstruct([server_payload], [shared_data], server.secrets, 
                                                      dryrun=cfg.dryrun)

Next we'll evaluate metrics, comparing the `reconstructed_user_data` to the `true_user_data`.

In [ ]:
metrics = breaching.analysis.report(reconstructed_user_data, true_user_data, [server_payload], 
                                    server.model, order_batch=True, compute_full_iip=False, 
                                    cfg_case=cfg.case, setup=setup)

And finally, we also plot the reconstructed data:

In [ ]:
user.print(reconstructed_user_data)

### Notes:
* Choose whatever text you want ;>
* The segment in this notebook has 11k tokens, so it is quite a bit longer than the example from the paper. You can boost the accuracy by making it shorter again.
* This text is recovered based only on some basic statistical analysis. 70% of the text is recovered perfectly, the rest is scrambled. The scrambled tokens could have been unscrambled further by using an actual language model to score likely sentences, but we wanted to see the performance of the "raw" output in this work, without contamination from a language model, which would be biased toward "likely" sentences.
* The recovery in this notebook is entirely data-agnostic, it would work about as well for a stream of random tokens.